In [16]:
from my_import import *
df_train = pd.read_csv('df_train.csv')
df_val = pd.read_csv('df_val.csv')
df_test = pd.read_csv('df_test.csv')
final_cleaned_dataset_df=pd.read_csv('final_cleaned_dataset_df.csv')

#Make sure the genre collumns is in lists not strings
#NEED TO DO THIS EVERYTIME EXPORT DATASET
df_train['genres'] = df_train['genres'].apply(lambda x: list(ast.literal_eval(x)))
df_val['genres'] = df_val['genres'].apply(lambda x: list(ast.literal_eval(x)))
df_test['genres'] = df_test['genres'].apply(lambda x: list(ast.literal_eval(x)))
final_cleaned_dataset_df['genres'] = final_cleaned_dataset_df['genres'].apply(lambda x: list(ast.literal_eval(x)))

In [17]:
# import nltk
# from nltk.stem import WordNetLemmatizer
# from nltk.corpus import stopwords
# import re

# Download essential resources
# nltk.download('punkt_tab')         # tokenizer
# nltk.download('stopwords')     # stopword list
# nltk.download('wordnet')       # lemmatizer
# nltk.download('omw-1.4')       # lemmatizer support data
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def clean_text(text):
    tokens = nltk.word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalpha() and word not in stop_words]
    return ' '.join(tokens)

df_train['clean_synopsis'] = df_train['synopsis'].apply(clean_text)
df_val['clean_synopsis'] = df_val['synopsis'].apply(clean_text)
df_test['clean_synopsis'] = df_test['synopsis'].apply(clean_text)




In [18]:
from collections import defaultdict
import pandas as pd

# Example structure: genres = ['Action', 'Comedy'], synopsis = 'He fights for justice in a dystopian world.'
genre_keywords = defaultdict(list)

# Tokenize and associate words with genres
for _, row in df_train.iterrows():
    words = row['clean_synopsis'].lower().split()  # crude tokenization
    for genre in row['genres']:  # assuming genres is a stringified list
        genre_keywords[genre].extend(words)


In [22]:
type(genre_keywords)

collections.defaultdict

In [25]:

from collections import Counter

top_words_per_genre = {}
for genre, words in genre_keywords.items():
    top_words = Counter(words).most_common(20)  # top 20 keywords
    top_words_per_genre[genre] = [word for word, _ in top_words]

print(top_words_per_genre)

{'Action': ['source', 'world', 'one', 'life', 'new', 'however', 'year', 'power', 'find', 'friend', 'human', 'two', 'girl', 'earth', 'school', 'battle', 'ann', 'city', 'mysterious', 'take'], 'Adventure': ['source', 'world', 'one', 'life', 'find', 'new', 'friend', 'ann', 'however', 'year', 'power', 'earth', 'time', 'girl', 'two', 'must', 'mysterious', 'young', 'take', 'human'], 'Comedy': ['source', 'school', 'girl', 'one', 'life', 'new', 'world', 'episode', 'friend', 'however', 'ann', 'find', 'day', 'time', 'student', 'series', 'high', 'year', 'story', 'two'], 'Kids': ['source', 'world', 'friend', 'new', 'child', 'series', 'story', 'anime', 'one', 'pokémon', 'boy', 'find', 'life', 'ann', 'time', 'short', 'named', 'film', 'pokemon', 'take'], 'Sci-Fi': ['source', 'world', 'earth', 'one', 'year', 'new', 'life', 'ann', 'human', 'however', 'girl', 'planet', 'time', 'find', 'friend', 'known', 'robot', 'power', 'mysterious', 'city'], 'Sports': ['team', 'school', 'source', 'high', 'new', 'player

In [26]:

keyword_to_genres = defaultdict(set)

for genre, words in top_words_per_genre.items():
    for word in words:
        keyword_to_genres[word].add(genre)
        


In [29]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
mlb.fit(df_val['genres'])  # fit on all ground-truth labels

MultiLabelBinarizer()

In [33]:

def heuristic_predict(synopsis):
    tokens = synopsis.lower().split()
    predicted = set()
    for token in tokens:
        if token in keyword_to_genres:
            predicted.update(keyword_to_genres[token])
    return list(predicted) if predicted else ["Unknown"]  # or return most common genres

from sklearn.metrics import f1_score

y_true = []
y_pred = []

subject=df_test

for _, row in subject.iterrows():
    pred = heuristic_predict(row['synopsis'])
    true = row['genres']

    y_pred.append(pred)
    y_true.append(true)


subject["predicted genres"]=y_pred
display(subject[['synopsis','genres','predicted genres']])
# Convert to multi-hot encoding if needed for metric calculation
y_true_bin = mlb.transform(y_true)
y_pred_bin = mlb.transform(y_pred)

from sklearn.metrics import f1_score, precision_score, recall_score

f1 = f1_score(y_true_bin, y_pred_bin, average='micro')  # or 'macro', 'samples'
precision = precision_score(y_true_bin, y_pred_bin, average='micro')
recall = recall_score(y_true_bin, y_pred_bin, average='micro')

print(f"F1 Score: {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

,synopsis,genres,predicted genres
0,The story begins 15 years after the demon king...,"[Comedy, Fantasy, Slice of Life]","[Historical, Drama, Super Power, Romance, Shou..."
1,A cyborg warrior from an ancient Antarctic kin...,"[Action, Mecha, Sci-Fi]","[Historical, Drama, Super Power, Kids, Romance..."
2,"The story centers on Otarou, a creature themed...",[Comedy],"[Historical, Drama, Kids, Romance, Parody, Sho..."
3,"An elite soldier, whose nickname is God of Dea...","[Action, Adventure, Comedy, Romance, Sci-Fi]","[Military, Sci-Fi, Mecha]"
4,"Based on the real life-story of Pink Lady, a f...","[Drama, Music, Slice of Life]",[Hentai]
...,...,...,...
1122,An unaired episode released with the sixth BD ...,"[Sci-Fi, Seinen]","[Parody, Slice of Life, School, Harem, Comedy,..."
1123,Short extras of the anime included in each of ...,[Romance],"[Parody, Kids, Music]"
1124,Short specials included in the Blu-ray release...,"[Sci-Fi, Seinen]","[Parody, Kids, Music]"
1125,Three episodes aired on TV as part of the Ribo...,[Shoujo],"[Drama, Super Power, Romance, Shounen, Shoujo,..."


F1 Score: 0.2208
Precision: 0.1271
Recall: 0.8369


c:\Users\User\anaconda3\envs\nlp_env\lib\site-packages\sklearn\preprocessing\_label.py:909: UserWarning: unknown class(es) ['Unknown'] will be ignored
  warnings.warn(
